# Análise de Fontes para Gerar a Regra das Exceções no Processo de Encerramento

In [80]:
import pandas as pd
import numpy as np

# --- Formatação de exibição (opcional) ---
pd.options.display.float_format = lambda x: f'{x:,.2f}'.replace(',', 'X').replace('.', ',').replace('X', '.')

In [81]:
arquivo = "Diferenças entre C_C 72111 x 82114 e 82115.csv"

In [82]:
# Leitura do CSV com separador e tipos definidos
df = pd.read_csv(
    arquivo,          # seu CSV
    sep=";",
    dtype=str,       # códigos como str, valores como float
    encoding="latin-1",  # ou "latin-1"
)

df = df.drop(index=0)
df = df.rename(columns={
    'Unidade Gestora / Conta Corrente': 'conta_corrente',
    'Conta 721110101 (A)': 'conta_721',
    'Contas 82114 (B)': 'conta_82114',
    'Contas 82115 (C)': 'conta_82115',
    'Diferença = (A) - (B) - (C) ': 'dif_dispon' # cuidado que tem espaço no final da coluna

})


# --- Lista de colunas numéricas ---
COLS_VAL = [
    'conta_721',
    'conta_82114',
    'conta_82115',
    'dif_dispon',
]

# --- Função de conversão PT-BR → float ---
def to_float_ptbr(s):
    return pd.to_numeric(
        s.astype(str)
         .str.strip()
         .str.replace(r'\s+', '', regex=True)      # remove espaços
         .str.replace(r'^\((.*)\)$', r'-\1', regex=True)  # (123,45) -> -123,45
         .str.replace('.', '', regex=False)        # remove milhar
         .str.replace(',', '.', regex=False),      # vírgula -> ponto
        errors='coerce'
    )

# --- Conversão das colunas de valores para float ---
for col in COLS_VAL:
    df[col] = to_float_ptbr(df[col])


df = df[~((df['conta_82114'] == 0) & (df['conta_82115'] == 0))]

# 6ª coluna (como você já tem)
df['1_processo_82115'] = df['conta_721'] - df['conta_82115']

# 7ª coluna com lógica condicional baseada no sinal da 6ª
df['2_processo_82114'] = np.where(
    df['1_processo_82115'] >= 0,
    df['1_processo_82115'] - df['conta_82114'],  # caso positivo/zero
    df['conta_721'] - df['conta_82114']          # caso negativo
)


df

,conta_corrente,conta_721,conta_82114,conta_82115,dif_dispon,1_processo_82115,2_processo_82114
1,1.5.00.100.0.000000,"215.210.307.505,41","108.490.082.675,50","110.103.589.225,45","-3.383.364.395,54","105.106.718.279,96","-3.383.364.395,54"
2,1.5.00.100.1.000001,"0,00","0,00","1.693,54","-1.693,54","-1.693,54","0,00"
3,1.5.00.100.1.002017,"0,00","0,00","87.213.311,43","-87.213.311,43","-87.213.311,43","0,00"
4,1.5.00.100.1.007017,"0,00","0,00","7.848,00","-7.848,00","-7.848,00","0,00"
5,1.5.00.100.1.007526,"0,00","24.655.036,74","0,00","-24.655.036,74","0,00","-24.655.036,74"
...,...,...,...,...,...,...,...
1562,2.7.59.152.2.007289,"5.742,67","0,00","5.742,67","0,00","0,00","0,00"
1563,2.8.62.081.0.000000,"447.679,07","29.832,95","163.949,65","253.896,47","283.729,42","253.896,47"
1564,2.8.69.188.0.000000,"331.998,15","484.166,73","0,00","-152.168,58","331.998,15","-152.168,58"
1565,2.8.98.085.0.000000,"-362.415,04","0,00","-510.369,55","147.954,51","147.954,51","147.954,51"


In [83]:
df['status_proc1'] = np.where(df['1_processo_82115'] < 0, 'NEGATIVO', 'POSITIVO')
df['status_proc2'] = np.where(df['2_processo_82114'] < 0, 'NEGATIVO', 'POSITIVO')

# Subconjuntos para análise (só negativos)
df_proc1_neg = df[df['1_processo_82115'] < 0].copy()
df_proc2_neg = df[df['2_processo_82114'] < 0].copy()


In [90]:
regra_conta_82115 = df_proc1_neg.copy()

# Garantir tipo string e remover pontos
regra_conta_82115['conta_corrente_limpa'] = (
    regra_conta_82115['conta_corrente']
    .astype(str)
    .str.replace('.', '', regex=False)
    .str.strip()
)

# Criar as duas novas colunas 
regra_conta_82115['ano_fonte'] = regra_conta_82115['conta_corrente_limpa'].str[:1]
regra_conta_82115['FONTE_STN'] = regra_conta_82115['conta_corrente_limpa'].str[1:4]
regra_conta_82115['detalhamento'] = regra_conta_82115['conta_corrente_limpa'].str[-6:]

regra_conta_82115 = regra_conta_82115[['ano_fonte', 'FONTE_STN', 'detalhamento', 'conta_corrente_limpa']]

regra_conta_82115

,ano_fonte,FONTE_STN,detalhamento,conta_corrente_limpa
2,1,500,000001,15001001000001
3,1,500,002017,15001001002017
4,1,500,007017,15001001007017
9,1,500,990025,15001001990025
40,1,500,006056,15001002006056
...,...,...,...,...
1169,1,761,210090,17611222210090
1401,1,898,000000,18980850000000
1419,1,999,006623,19992122006623
1442,1,999,000000,19992240000000


In [93]:
regra_conta_82115.query('detalhamento == "000000"')

,ano_fonte,FONTE_STN,detalhamento,conta_corrente_limpa
688,1,757,000000,17571900000000
1401,1,898,000000,18980850000000
1442,1,999,000000,19992240000000
1497,2,500,000000,25001000000000


In [94]:
regra_conta_82115.query('ano_fonte == "1" and FONTE_STN == "500"')

,ano_fonte,FONTE_STN,detalhamento,conta_corrente_limpa
2,1,500,000001,15001001000001
3,1,500,002017,15001001002017
4,1,500,007017,15001001007017
9,1,500,990025,15001001990025
40,1,500,006056,15001002006056
...,...,...,...,...
325,1,500,200023,15001002200023
329,1,500,200042,15001002200042
341,1,500,210093,15001002210093
344,1,500,210138,15001002210138


In [92]:
regra_conta_82114 = df_proc2_neg.copy()

# Garantir tipo string e remover pontos
regra_conta_82114['conta_corrente_limpa'] = (
    regra_conta_82114['conta_corrente']
    .astype(str)
    .str.replace('.', '', regex=False)
    .str.strip()
)

# Criar as duas novas colunas 
regra_conta_82114['ano_fonte'] = regra_conta_82114['conta_corrente_limpa'].str[:1]
regra_conta_82114['FONTE_STN'] = regra_conta_82114['conta_corrente_limpa'].str[1:4]
regra_conta_82114['detalhamento'] = regra_conta_82114['conta_corrente_limpa'].str[-6:]

regra_conta_82114 = regra_conta_82114[['ano_fonte', 'FONTE_STN', 'detalhamento', 'conta_corrente_limpa']]

regra_conta_82114

,ano_fonte,FONTE_STN,detalhamento,conta_corrente_limpa
1,1,500,000000,15001000000000
5,1,500,007526,15001001007526
6,1,500,047928,15001001047928
8,1,500,705582,15001001705582
176,1,500,007289,15001002007289
...,...,...,...,...
1539,2,501,210128,25011452210128
1540,2,501,210170,25011452210170
1541,2,501,220015,25011452220015
1542,2,501,230020,25011452230020


In [95]:
regra_conta_82114.query('detalhamento == "000000"')

,ano_fonte,FONTE_STN,detalhamento,conta_corrente_limpa
1,1,500,000000,15001000000000
394,1,500,000000,15001070000000
473,1,501,000000,15011200000000
634,1,754,000000,17541110000000
688,1,757,000000,17571900000000
1401,1,898,000000,18980850000000
1497,2,500,000000,25001000000000
1502,2,501,000000,25011200000000
1564,2,869,000000,28691880000000


In [96]:
regra_conta_82114.query('ano_fonte == "1" and FONTE_STN == "500"')

,ano_fonte,FONTE_STN,detalhamento,conta_corrente_limpa
1,1,500,000000,15001000000000
5,1,500,007526,15001001007526
6,1,500,047928,15001001047928
8,1,500,705582,15001001705582
176,1,500,007289,15001002007289
...,...,...,...,...
419,1,500,250001,15001482250001
420,1,500,250023,15001482250023
421,1,500,250052,15001482250052
422,1,500,250062,15001482250062
